In [1]:
import numpy as np
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_boston,load_iris

In [2]:
lb = load_boston()
features = lb.feature_names
x_reg ,y_reg = lb.data,lb.target
x_reg = pd.DataFrame(x_reg,columns=features)
y_reg = pd.DataFrame(y_reg,columns=['target'])

In [3]:
li = load_iris()
features2 = li.feature_names
x_cls ,y_cls = li.data,li.target
x_cls = pd.DataFrame(x_cls,columns=features2)
y_cls = pd.DataFrame(y_cls,columns=['target'])

In [4]:
x_reg.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [5]:
x_cls.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,accuracy_score,mean_absolute_error,roc_auc_score,mean_squared_error
from sklearn.linear_model import LinearRegression

## 线性回归

普通最小二乘的系数估计依赖于**特征的独立性**。

当特征相关且设计矩阵的列之间具有**近似线性相关性**时， 设计矩阵趋于**奇异矩阵**，最小二乘估计对观测目标的随机误差高度敏感，可能产生很大的方差。

R ^ 2定义为（1- u / v），其中u是残差平方和（（y_true-y_pred）** 2）.sum（），而v是总平方和（（y_true- y_true.mean（））** 2）.sum（）（也叫真实值的离差平方和）;

[线性模型系数解释中的常见缺陷](http://scikit-learn.org.cn/view/257.html)

In [7]:
X_reg_train,X_reg_test,y_reg_train,y_reg_test = train_test_split(x_reg,y_reg,random_state=2022,test_size=0.2)

'''
normalize=True , 在回归之前通过减去均值并除以l2范数来对回归变量X进行归一化；
如果不在LinearRegression对象里调用归一化，则使用sklearn.preprocessing.StandardScaler
'''
reg = LinearRegression(normalize=True) 

# 通过 sample_weight 的平方根重新缩放数据样本
reg.fit(X_reg_train,y_reg_train,sample_weight=0.25)
print(f'系数：{reg.coef_} \n 截距：{reg.intercept_}')
print(f'训练集的r2:{reg.score(X_reg_train,y_reg_train)}')

y_reg_pred = reg.predict(X_reg_test)
print(f'测试集的r2:{r2_score(y_reg_test,y_reg_pred)}')

系数：[[-1.09358524e-01  4.17604608e-02  4.92275845e-02  2.75483861e+00
  -1.37425402e+01  4.71242684e+00 -1.19656681e-02 -1.43522444e+00
   2.98673854e-01 -1.23637115e-02 -8.95540107e-01  1.03495304e-02
  -5.21139480e-01]] 
 截距：[27.61502321]
训练集的r2:0.7560832029228641
测试集的r2:0.6225687597000795


## 岭回归 Ridge

线性回归基础上 + **L2正则项**

![20220706210934](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220706210934.png)

In [8]:
from sklearn.linear_model import Ridge

In [9]:
reg_ridge = Ridge(alpha=0.5,normalize=True) # alpha 为正则化系数，越大，则惩罚越强，w越小
reg_ridge.fit(X_reg_train,y_reg_train)
print(f'系数：{reg_ridge.coef_}')


print(f'训练集的r2:{reg_ridge.score(X_reg_train,y_reg_train)}')

y_reg_pred = reg_ridge.predict(X_reg_test)
print(f'测试集的r2:{r2_score(y_reg_test,y_reg_pred)}')

系数：[[-6.86575752e-02  1.83801716e-02 -5.51674004e-02  2.71481628e+00
  -4.30433161e+00  4.04915189e+00 -9.39472663e-03 -4.92180619e-01
   2.76924906e-02 -2.87975920e-03 -6.44100560e-01  7.47061607e-03
  -3.44996611e-01]]
训练集的r2:0.7033850851955237
测试集的r2:0.6452418447538283


## 岭回归变种 RidgeClassifier

也叫，带有线性核的最小二乘支持向量机

使用Ridge回归的分类器，首先**将目标值转换为{-1, 1}**，然后将问题视为回归任务（在多类情况下为多输出回归）。

**预测类对应于回归预测的符号**

In [10]:
from sklearn.linear_model import RidgeClassifier

X_cls_train,X_cls_test,y_cls_train,y_cls_test = train_test_split(x_cls,y_cls,random_state=2022,test_size=0.2)

cls_ridge = RidgeClassifier(normalize=True)
cls_ridge.fit(X_cls_train,y_cls_train)
cls_ridge.coef_
print(f'训练集准确率:{cls_ridge.score(X_cls_train,y_cls_train)}') 


y_cls_pred = cls_ridge.predict(X_cls_test)
print(f'测试集准确率：{accuracy_score(y_cls_test,y_cls_pred)}')

训练集准确率:0.8583333333333333
测试集准确率：0.9


![20220706234704](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220706234704.png)

In [11]:
# 预测样本的置信度分数
# 样本的置信度分数与该样本到超平面的有符号距离成正比
confidence_score = cls_ridge.decision_function([[4.5,3.1,2.0,0.35]])
print(confidence_score)

print(f'属于第{np.argmax(confidence_score)}类')

[[ 0.38820477 -0.40871633 -0.97948845]]
属于第0类


## RidgeCV 

RidgeCV 通过内置的alpha 参数的控制交叉验证来实现岭回归。 

该对象与 GridSearchCV 的使用方法相同，如果不指定cv（cv=None），默认为留一交叉验证方法（LOO-CV，即抽取一个样本作为测试集数据）

In [12]:
from sklearn.linear_model import RidgeCV

reg_ridgecv = RidgeCV(alphas=np.logspace(-6,6,13),cv=10) 
reg_ridgecv.fit(X_reg_train,y_reg_train)
print(f'最佳alpha取值：{reg_ridgecv.alpha_}')
reg_ridgecv.coef_


print(f'训练集的r2:{reg_ridgecv.score(X_reg_train,y_reg_train)}')

y_reg_pred = reg_ridgecv.predict(X_reg_test)
print(f'测试集的r2:{r2_score(y_reg_test,y_reg_pred)}')

最佳alpha取值：1.0
训练集的r2:0.7547330246063915
测试集的r2:0.6120292876368366


## Lasso

Lasso是一个估计稀疏系数的线性模型。它在某些情况下是有用的，因为它倾向于给出非零系数较少的解，从而有效地减少了给定解所依赖的特征数。

线性回归 + L1正则项

![20220707002437](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220707002437.png)

Lasso产生稀疏矩阵，可用来作特征选择，alpha参数越高，所选择的特征就越少。

In [13]:
from sklearn.linear_model import  Lasso, LassoCV

reg_lasso = Lasso(alpha=1.1)
reg_lasso.fit(X_reg_train,y_reg_train)

print(f'系数：{np.round(reg_lasso.coef_,5)} \n 截距：{reg_lasso.intercept_}')
print(f'训练集的r2:{reg.score(X_reg_train,y_reg_train)}')

y_reg_pred = reg_lasso.predict(X_reg_test)
print(f'测试集的r2:{r2_score(y_reg_test,y_reg_pred)}')

系数：[-0.06458  0.04446 -0.       0.      -0.       1.13055  0.02012 -0.71969
  0.27929 -0.01466 -0.7708   0.00871 -0.80275] 
 截距：[41.15011875]
训练集的r2:0.7560832029228641
测试集的r2:0.6460796009959273


In [25]:
reg_lassocv = LassoCV(cv=5,random_state=2022,alphas=np.arange(0,2,0.01))
reg_lassocv.fit(X_reg_train,y_reg_train)
print(f'最佳alpha：{reg_lassocv.alpha_}')

print(f'系数：{np.round(reg_lassocv.coef_,5)} \n 截距：{reg_lassocv.intercept_}')
print(f'训练集score：{reg_lassocv.score(X_reg_train,y_reg_train)}')

y_reg_pred = reg_lassocv.predict(X_reg_test)
print(f'测试集score：{r2_score(y_reg_test,y_reg_pred)}')

最佳alpha：0.0
系数：[-1.093600e-01  4.176000e-02  4.923000e-02  2.754840e+00 -1.374254e+01
  4.712430e+00 -1.197000e-02 -1.435220e+00  2.986700e-01 -1.236000e-02
 -8.955400e-01  1.035000e-02 -5.211400e-01] 
 截距：27.615023209546624
训练集score：0.7560832029228641
测试集score：0.6225687597000796


## 弹性网络 -- ElasticNet 

通过调整**l1_ratio**，调整L1\L2的组合

![20220707101826](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220707101826.png)


In [31]:
from sklearn.linear_model import ElasticNet

reg_elsnet = ElasticNet(alpha=0.5,l1_ratio=0.5)
reg_elsnet.fit(X_reg_train,y_reg_train)

print(f'系数：{np.round(reg_elsnet.coef_,5)} \n 截距：{reg_elsnet.intercept_}')
print(f'训练集r2：{reg_elsnet.score(X_reg_train,y_reg_train)}')

y_reg_pred = reg_elsnet.predict(X_reg_test)
print(f'测试集r2:{r2_score(y_reg_test,y_reg_pred)}')

系数：[-0.09379  0.05137 -0.       0.      -0.       2.01367  0.00582 -1.05469
  0.33827 -0.01671 -0.85229  0.0097  -0.7472 ] 
 截距：[38.58675094]
训练集r2：0.7200146013972221
测试集r2:0.6388913772483735


## 最小角回归 -- Least-angle regression， LARS

用于高维数据的回归算法。

LARs和逐步向前回归很相似。在每一步中，它都会找到与目标最相关的特征。当多个特征具有相等的相关性时，它不是沿着相同的特征继续进行，而是沿着特征之间等角的方向进行。

![20220707110207](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220707110207.png)

In [40]:
from sklearn.linear_model import Lars

# n_nonzero_coefs , 非0系数的数量
reg_lars = Lars(n_nonzero_coefs=10)
reg_lars.fit(X_reg_train,y_reg_train)
print(reg_lars.coef_ )
print(f'训练集r2:{reg.score(X_reg_train,y_reg_train)}')

y_reg_pred = reg_lars.predict(X_reg_test)
print(f'测试集r2:{r2_score(y_reg_test,y_reg_pred)}')

[-0.0582612   0.01447872  0.          2.54782975 -7.4947512   5.06727161
  0.         -0.81838349  0.01354248  0.         -0.79341687  0.00801916
 -0.52311074]
训练集r2:0.7560832029228641
测试集r2:0.6069215238656641


## 贝叶斯回归

贝叶斯回归技术可以用于**预估正则化参数**：

正则化参数不是在硬意义上设置，而是根据手头的数据进行调整。(通过在模型的超参数上引入信息不足的先验（uninformative priors）来实现。)


贝叶斯回归的优点是：

- 它可以调整以适应手头的数据
- 它可以用于在估计过程中引入正则项。

贝叶斯回归的缺点包括：
- 模型的推断可能很费时间。


In [42]:
# 贝叶斯岭回归
from sklearn.linear_model import BayesianRidge

reg_bayesridge = BayesianRidge()
reg_bayesridge.fit(X_reg_train,y_reg_train)
print(reg_bayesridge.coef_)

print(reg_bayesridge.score(X_reg_train,y_reg_train))

[-1.04920198e-01  4.58620000e-02  8.24472523e-04  1.93876881e+00
 -1.68585850e+00  4.40793413e+00 -1.83356680e-02 -1.26773465e+00
  2.89327632e-01 -1.40982365e-02 -7.97718311e-01  1.07600296e-02
 -5.71063634e-01]
0.7499565296829631


## Logistic回归

![20220707132351](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220707132351.png)

“lbfgs”鲁棒性更强， 默认是“lbfgs”。

对于大型数据集，“saga”通常更快。对于大型数据集，还可考虑使用对数损失( ‘log’ loss)的 SGDClassifier，这可能会更快，但需要更多的调优。

In [59]:
from sklearn.linear_model import LogisticRegression

# C值为正则强度的倒数 
# 所以，较大的C值给了模型更多的自由度。相反，较小的C值指定更强的正则化
cls_lr = LogisticRegression(penalty='l1',tol=1e-6,solver='liblinear',C=0.7)
# cls_lr = LogisticRegression(l1_ratio=1,tol=1e-6,solver='liblinear',C=0.7) # 二者等价；l1_ratio=0，则是penalty='l2'
cls_lr.fit(X_cls_train,y_cls_train)

print(f'决策函数特征系数:\n{cls_lr.coef_}')
print(f'训练集准确度：{cls_lr.score(X_cls_train,y_cls_train)}')

# 预测类别标签
y_cls_pred = cls_lr.predict(X_cls_test)
print(f'测试集准确率：{accuracy_score(y_cls_test,y_cls_pred)}')

# 预测每个样本属于每类的概率估计
for i in cls_lr.predict_proba(X_cls_test)[:5]:
    print(np.argmax(i))

决策函数特征系数:
[[ 0.          2.17392719 -2.46070693  0.        ]
 [ 0.39416871 -1.25458345  0.56418343 -1.09830101]
 [-1.66201155 -2.23765367  1.85357187  4.59524288]]
训练集准确度：0.975
测试集准确率：0.9
2
2
0
2
0


In [61]:
y_cls_pred[:5]

array([2, 2, 0, 2, 0])

## 随机梯度下降-- SGD

严格地说，SGD只是一种优化技术，并不与特定的机器学习模型相对应。它只是**训练模型的一种方式**；

例如，使用`SGDClassifier(loss='log')`将导致Logistic回归， 即`等价于LogisticRegression的模型可以通过SGD拟合`，而不是LogisticRegression中其他的优化方案。

SGDRegressor(loss='squared_loss', penalty='l2')和 Ridge就是通过不同的方法解决了相同的优化问题。

- 优点：实现简单、速度贼快；
- 缺点：**对特征值的量纲贼敏感，一定要提前进行特征缩放**，不然梯度下降成“Z”字型，瞎晃~~~（相同的缩放必须应用于测试向量以获得有意义的结果）

`在拟合模型之前，一定要重新排序(打乱的)训练数据，或者在每次迭代后(默认情况下使用)使用shuffle=True来打乱数据`

分类：

![20220707142556](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220707142556.png)

回归：

![20220707142703](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220707142703.png)

In [83]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

# 特征缩放
scaler = StandardScaler()
x_cls_scale = scaler.fit_transform(li.data) 

x_cls_trn_scale , x_cls_test_scale , y_cls_train,y_cls_test = train_test_split(x_cls_scale,y_cls,random_state=2022,test_size=0.2)

# SGD版本的LR
sgdcls = SGDClassifier(loss='log',penalty='l1',l1_ratio=0.5,shuffle=True)
sgdcls.fit(x_cls_trn_scale,y_cls_train)
print(f'训练集准确率：{sgdcls.score(x_cls_trn_scale,y_cls_train)}')

y_cls_pred = sgdcls.predict(x_cls_test_scale)
print(f'测试集准确率：{accuracy_score(y_cls_test,y_cls_pred)}')

sgdcls.predict_proba(x_cls_test_scale)[:5]

训练集准确率：0.975
测试集准确率：0.9333333333333333


array([[1.67330241e-36, 3.21921928e-01, 6.78078072e-01],
       [2.71639726e-50, 4.60960723e-01, 5.39039277e-01],
       [7.92078088e-01, 2.07921912e-01, 9.92394448e-60],
       [2.47199837e-31, 4.04092318e-01, 5.95907682e-01],
       [9.77401844e-01, 2.25981563e-02, 1.37137866e-63]])

附：

![20220707143203](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220707143203.png)